In [ ]:
!pip install langchain langchain_google_genai python-dotenv langchain_community -q
!pip install pymupdf easyocr pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
from google.colab import userdata
GOOGLE_GENAI_KEY = userdata.get('GOOGLE_GENAI_KEY')

In [ ]:
import google.generativeai as genai
genai.configure(api_key=GOOGLE_GENAI_KEY)

Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko

In [ ]:
import fitz  # PyMuPDF
import easyocr
from PIL import Image
import io

# Initialize EasyOCR reader (English by default, add other languages if needed)
reader = easyocr.Reader(['en'])  # Example: ['en', 'de'] for English & German

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = []

    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text("text")

        if text.strip():
            full_text.append(text)
        else:
            image_list = page.get_images(full=True)
            extracted_text = ""

            for img_index, img in enumerate(image_list):
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]

                img_pil = Image.open(io.BytesIO(image_bytes))
                extracted_text += "\n".join(reader.readtext(image_bytes, detail=0)) + "\n"

            full_text.append(extracted_text)

    return "\n".join(full_text)


# Example Usage:
doc_path = r"/content/2025021580.pdf"
extracted_text = extract_text_from_pdf(doc_path)

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_GENAI_KEY)

In [ ]:
message = f"""
[Summarization]
You are a summarization model under testing.
Only use the give document below to answer the query.
{extracted_text}
"""

# Invoke the model
response = model.invoke(message)

# Print the response
print(response.content)

The main goal of the book is to provide an intuition into the field of LLMs.


below is use GPU with easyocr (its an deep learning based OCR build with pytorch)

In [ ]:
import fitz  # PyMuPDF
import easyocr
from PIL import Image
import io
import torch

# Check if GPU is available
use_gpu = torch.cuda.is_available()

# Initialize EasyOCR reader (English only, add other languages if needed)
reader = easyocr.Reader(['en'], gpu=use_gpu)  # Enable GPU if available

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = []

    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text("text")

        if text.strip():
            full_text.append(text)
        else:
            image_list = page.get_images(full=True)
            extracted_text = ""

            for img_index, img in enumerate(image_list):
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]

                img_pil = Image.open(io.BytesIO(image_bytes))
                extracted_text += "\n".join(reader.readtext(image_bytes, detail=0)) + "\n"

            full_text.append(extracted_text)

    return "\n".join(full_text)


# Example Usage:
doc_path = r"/content/2025021580.pdf" 
extracted_text = extract_text_from_pdf(doc_path)